# 데이터 불러오기

In [108]:
# from google.colab import drive
# drive.mount('/content/drive')

- 시드값

In [1]:
SEED = 42

- 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Data/"
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

# 정답 데이터

In [3]:
target = train_target["target"]

# cv 점수 확인해보기

## 다중 레이어 앙상블

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 데이터 준비
X_train = ...
y_train = ...
X_test = ...

# 1단계: 기본 모델 정의
base_models = [
    LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=6),
    XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, use_label_encoder=False, eval_metric='logloss'),
    CatBoostClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, verbose=0)
]

# 2단계: 중간 모델 정의
meta_model_1 = LogisticRegression()

# 3단계: 최종 모델 정의
meta_model_2 = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# 교차 검증 및 예측 저장
n_folds = 5
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# 1단계: Base models 학습 및 예측
base_model_preds = np.zeros((X_train.shape[0], len(base_models)))
test_preds_stage1 = np.zeros((X_test.shape[0], len(base_models)))

for i, model in enumerate(base_models):
    for train_idx, val_idx in kf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        model.fit(X_tr, y_tr)
        base_model_preds[val_idx, i] = model.predict_proba(X_val)[:, 1]  # Class 1 확률
        test_preds_stage1[:, i] += model.predict_proba(X_test)[:, 1] / n_folds  # Test 평균 예측

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, y_train)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), y_train)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(y_train, (meta_preds_stage2 > 0.5).astype(int)))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 데이터 준비
# X_train = train_ft
# y_train = target
# X_test = test_ft

# 1단계: 기본 모델 정의
base_models = [
    LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=6),
    XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, use_label_encoder=False, eval_metric='logloss'),
    CatBoostClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, verbose=0)
]

# 2단계: 중간 모델 정의
meta_model_1 = LogisticRegression()

# 3단계: 최종 모델 정의
meta_model_2 = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# 1단계: Base models 학습 및 예측
base_model_preds = np.zeros((train_ft.shape[0], len(base_models)))
test_preds_stage1 = np.zeros((test_ft.shape[0], len(base_models)))

for i, model in enumerate(base_models):
    for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
        x_train = train_ft.iloc[tri]
        y_train = target.iloc[tri]
        x_valid = train_ft.iloc[vai]
        y_valid = target.iloc[vai]
        
        model.fit(x_train, y_train)
        base_model_preds[vai, i] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
        test_preds_stage1[:, i] += model.predict_proba(test_ft)[:, 1] / n_folds  # Test 평균 예측

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.5).astype(int)))


## 실험

### 1단계 

In [16]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_군집_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_군집_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1471), (12225, 1471))

In [17]:
target = train_target["target"]

In [18]:
base_model_preds = np.zeros((train_ft.shape[0], 3))
test_preds_stage1 = np.zeros((test_ft.shape[0], 3))

base_model_preds.shape, test_preds_stage1.shape

((14940, 3), (12225, 3))

In [19]:
base_model_preds, test_preds_stage1

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

- XGB_v3.0

In [24]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.0_1106.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.0_1106.csv")

train_ft.shape , test_ft.shape

((14940, 1504), (12225, 1504))

In [25]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1503), (12225, 1503))

In [26]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    246
dtype: int64

In [27]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1507), (12225, 1507))

In [28]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1509), (12225, 1509))

In [29]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1506), (12225, 1506))

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [32]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 0] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 0] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 0] = test_preds_stage1[:, 0] / n_folds

In [33]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.        , 0.        ],
        [0.91239369, 0.        , 0.        ],
        [0.06883854, 0.        , 0.        ],
        ...,
        [0.09131553, 0.        , 0.        ],
        [0.88809782, 0.        , 0.        ],
        [0.44763535, 0.        , 0.        ]]),
 array([[0.05952867, 0.        , 0.        ],
        [0.3970897 , 0.        , 0.        ],
        [0.3032796 , 0.        , 0.        ],
        ...,
        [0.03583201, 0.        , 0.        ],
        [0.13214782, 0.        , 0.        ],
        [0.0251264 , 0.        , 0.        ]]))

- XGB_v3.3

In [34]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_군집_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_군집_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1471), (12225, 1471))

In [35]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1470), (12225, 1470))

In [36]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [37]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1474), (12225, 1474))

In [38]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1476), (12225, 1476))

In [39]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [41]:
# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 1] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 1] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 1] = test_preds_stage1[:, 1] / n_folds

In [42]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.17354608, 0.        ],
        [0.91239369, 0.90294158, 0.        ],
        [0.06883854, 0.05948123, 0.        ],
        ...,
        [0.09131553, 0.05808163, 0.        ],
        [0.88809782, 0.8570658 , 0.        ],
        [0.44763535, 0.50293088, 0.        ]]),
 array([[0.05952867, 0.05414822, 0.        ],
        [0.3970897 , 0.370909  , 0.        ],
        [0.3032796 , 0.31456018, 0.        ],
        ...,
        [0.03583201, 0.03669401, 0.        ],
        [0.13214782, 0.11258472, 0.        ],
        [0.0251264 , 0.02188812, 0.        ]]))

- voting_v3.3_selection(logistic)

In [43]:
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,주구매_년,주구매_월,주구매_시간대,일별평균구매횟수,거래개월수,9시_12시_구매비율,12시_15시_구매비율,15시_18시_구매비율,18시_21시_구매비율,9시_12시_구매횟수,12시_15시_구매횟수,15시_18시_구매횟수,18시_21시_구매횟수,월초_구매비율,월말_구매비율,월초_구매횟수,월말_구매횟수,웨딩성수기_구매비율,웨딩성수기_구매횟수,1월_구매비율,2월_구매비율,3월_구매비율,4월_구매비율,5월_구매비율,6월_구매비율,7월_구매비율,8월_구매비율,9월_구매비율,10월_구매비율,...,공휴일_대분류_아동문화_총구매금액,공휴일_대분류_여성의류파트_총구매금액,공휴일_대분류_여성정장_총구매금액,공휴일_대분류_여성캐쥬얼_총구매금액,공휴일_대분류_영라이브_총구매금액,공휴일_대분류_영어덜트캐쥬얼_총구매금액,공휴일_대분류_영캐릭터_총구매금액,공휴일_대분류_영플라자_총구매금액,공휴일_대분류_잡화_총구매금액,공휴일_대분류_케주얼_구두_아동_총구매금액,공휴일_대분류_패션잡화_총구매금액,공휴일_대분류_가정용품_구매횟수,공휴일_대분류_골프_유니캐쥬얼_구매횟수,공휴일_대분류_공산품_구매횟수,공휴일_대분류_남성의류_구매횟수,공휴일_대분류_남성정장스포츠_구매횟수,공휴일_대분류_로얄부틱_구매횟수,공휴일_대분류_명품잡화_구매횟수,공휴일_대분류_생식품_구매횟수,공휴일_대분류_스포츠캐쥬얼_구매횟수,공휴일_대분류_아동_구매횟수,공휴일_대분류_아동_스포츠_구매횟수,공휴일_대분류_아동문화_구매횟수,공휴일_대분류_여성의류파트_구매횟수,공휴일_대분류_여성정장_구매횟수,공휴일_대분류_여성캐쥬얼_구매횟수,공휴일_대분류_영라이브_구매횟수,공휴일_대분류_영어덜트캐쥬얼_구매횟수,공휴일_대분류_영캐릭터_구매횟수,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,cluster,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.482151,1.851040,-1.345509,-0.057297,0.017620,-0.597009,0.704620,-0.413647,0.071850,-0.597009,0.704620,-0.413647,0.071850,0.567827,0.092192,-0.203556,-0.313132,-0.519468,-0.491469,-0.236981,-0.576086,-0.622120,-0.670149,-0.282014,0.421006,0.050936,-0.596434,1.047253,-0.698995,...,-0.086907,-0.092340,-0.062198,-0.134168,-0.095225,-0.068361,-0.081019,-0.126840,-0.084245,-0.174285,-0.162678,-0.222962,-0.150631,-0.301801,-0.179448,-0.139182,-0.105096,-0.225863,-0.252797,-0.165962,-0.110783,-0.148917,-0.121695,-0.128685,-0.138524,-0.211595,-0.116973,-0.108769,-0.10576,-0.162079,-0.286379,-0.209907,-0.213001,0.270695,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,0.109631,-0.482151,0.924134,1.132588,0.222706,1.176010,-0.231765,0.715412,-0.896636,0.333357,-0.231765,0.715412,-0.896636,0.333357,-0.429672,0.586200,-0.083161,0.523032,0.601863,0.544216,-0.256440,-0.576086,0.433398,-0.383294,0.579383,-0.500985,-0.521621,0.510469,1.370221,-0.328741,...,-0.086907,-0.092340,-0.062198,-0.134168,-0.095225,-0.068361,-0.081019,2.546882,-0.084245,-0.174285,3.590341,-0.222962,-0.150631,-0.301801,-0.179448,-0.139182,-0.105096,-0.225863,-0.252797,-0.165962,-0.110783,-0.148917,-0.121695,-0.128685,-0.138524,-0.211595,-0.116973,-0.108769,-0.10576,2.389509,-0.286379,-0.209907,4.161199,0.270695,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.482151,-0.620708,-0.106461,0.277707,1.465608,-0.126036,-0.102531,0.545699,-0.371335,-0.126036,-0.102531,0.545699,-0.371335,-0.412172,0.562013,1.180992,2.752802,0.483828,2.680316,0.214679,0.184729,0.220442,0.439526,0.579383,-0.290401,-0.239110,-0.351750,0.020979,-0.357972,...,-0.086907,-0.092340,-0.062198,-0.134168,-0.095225,-0.068361,-0.081019,-0.126840,-0.084245,-0.174285,-0.162678,-0.222962,2.490714,-0.301801,-0.179448,-0.139182,-0.105096,12.160452,-0.252797,-0.165962,-0.110783,2.274631,-0.121695,-0.128685,-0.138524,-0.211595,-0.116973,-0.108769,-0.10576,-0.162079,-0.286379,-0.209907,-0.213001,-3.694193,-0.604571,-0.623175,1.632592,-0.468181,-0.869072,1.064726
3,3.793345,-1.023980,-0.080558,0.080558,3.172703,4.429574,0.420933,-0.327474,0.008592,-0.135636,0.109631,-0.482151,-0.620708,0.719571,0.674668,1.465608,-0.160793,-0.089151,0.227983,-0.047873,-0.160793,-0.089151,0.227983,-0.047873,0.071441,-0.178796,4.190881,3.519286,0.081055,4.428034,-0.258285,0.163815,0.393378,0.300535,-0.056313,-0.120910,-0.023856,-0.464235,-0.005455,-0.183097,...,-0.086907,10.675203,-0.062198,-0.134168,-0.095225,-0.068361,-0.081019,8.611831,-0.084245,2.058189,-0.162678,-0.222962,-0.150631,-0.301801,-0.179448,5.151871,-0.105096,-0.225863,-0.252797,-0.165962,-0.110783,-0.148917,-0.1216

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

logi = LogisticRegression(random_state=42)
fs = SelectFromModel(logi)          # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
fs.fit_transform(train_ft, target)  # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환

best_cols_logi = fs.get_feature_names_out()

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
from sklearn.ensemble import VotingClassifier

params_xgb = {'n_estimators': 1000,'learning_rate': 0.027075406407767497,'max_depth': 5,'min_child_weight': 7,
              'subsample': 0.6789816859997232,'colsample_bytree': 0.6682531834544282,'gamma': 1.5046881781916308}
params_lgbm = {'n_estimators': 813, 'learning_rate': 0.014757440400599073, 'num_leaves': 35, 'max_depth': 12, 
               'min_child_samples': 41, 'subsample': 0.85, 'colsample_bytree': 0.95}
params_logi = {'C': 1.2056308154836568, 'solver': 'saga', 'max_iter': 400, 'class_weight': 'balanced', 
               'tol': 3.689794777633075e-05}            # optuna trial 30 : 0.727821596703826.

estimators = [
    ("lgbm", LGBMClassifier(random_state=42, **params_lgbm)),
    ("xgb", XGBClassifier(random_state=42, **params_xgb) ),
    ("logi", LogisticRegression(random_state=42, **params_logi) ),  ]

parmas = {
    "estimators": estimators,
    "voting" : "soft",
    "n_jobs" : -1   }

model = VotingClassifier(**parmas)

for tri, vai in cv.split(train_ft[best_cols_logi], target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft[best_cols_logi].iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft[best_cols_logi].iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 2] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 2] += model.predict_proba(test_ft[best_cols_logi])[:, 1]  

test_preds_stage1[:, 2] = test_preds_stage1[:, 2] / n_folds

In [46]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.17354608, 0.19652221],
        [0.91239369, 0.90294158, 0.83046801],
        [0.06883854, 0.05948123, 0.09595352],
        ...,
        [0.09131553, 0.05808163, 0.09557961],
        [0.88809782, 0.8570658 , 0.87621579],
        [0.44763535, 0.50293088, 0.67060904]]),
 array([[0.05952867, 0.05414822, 0.09066435],
        [0.3970897 , 0.370909  , 0.35863644],
        [0.3032796 , 0.31456018, 0.4260002 ],
        ...,
        [0.03583201, 0.03669401, 0.03895556],
        [0.13214782, 0.11258472, 0.43560855],
        [0.0251264 , 0.02188812, 0.01153557]]))

### 2단계

- 01

In [156]:
# 2단계: 중간 모델 정의
meta_model_1 = LogisticRegression()

# 3단계: 최종 모델 정의
meta_model_2 = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.5).astype(int)))

Final F1-Score: 0.6603418956412781


- 02

In [ ]:
# 2단계: 중간 모델 정의
lg_params = {'n_estimators': 323, 
             'criterion': 'entropy', 
             'max_depth': 4, 
             'num_leaves': 71, 
             'min_samples_split': 17, 
             'max_features': 0.8, 
             'bagging_fraction': 0.8154571896905384, 
             'bagging_freq': 3}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
meta_model_2 = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.4).astype(int)))

# 0.5 : 0.695031665328695
# 0.4 : 0.7173878586146775

Final F1-Score: 0.7173878586146775


- 03

In [70]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
meta_model_2 = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.4).astype(int)))

# 0.5 : 0.7068919281308662
# 0.4 : 0.7295865939451427

Final F1-Score: 0.7295865939451427


In [73]:
submit["target"] = (final_preds > 0.5).astype(int)
submit.to_csv("multi_layer_ensemble_03_1112.csv", index=False)

- 04

In [ ]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# 데이터 분리 (훈련 및 테스트 데이터 생성)
X_train, X_valid, y_train, y_valid = train_test_split(base_model_preds, target, test_size=0.2, random_state=42)

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
xgb_params = {'n_estimators': 1000,
            'learning_rate': 0.027075406407767497,
            'max_depth': 5,
            'min_child_weight': 7,
            'subsample': 0.6789816859997232,
            'colsample_bytree': 0.6682531834544282,
            'gamma': 1.5046881781916308}
meta_model_2 = XGBClassifier(**xgb_params)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(X_train, y_train)
meta_preds_stage2_train = meta_model_1.predict_proba(X_train)[:, 1]  # 학습 데이터 예측
meta_preds_stage2_valid = meta_model_1.predict_proba(X_valid)[:, 1]  # 검증 데이터 예측
meta_preds_stage2_test = meta_model_1.predict_proba(test_preds_stage1)[:, 1]  # 테스트 데이터 예측

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2_train.reshape(-1, 1), y_train)
final_preds_valid = meta_model_2.predict_proba(meta_preds_stage2_valid.reshape(-1, 1))[:, 1]
final_preds_test = meta_model_2.predict_proba(meta_preds_stage2_test.reshape(-1, 1))[:, 1]

# 검증 데이터 기반 최종 F1 점수 출력
threshold = 0.4  # 결정 임계값
final_f1_score = f1_score(y_valid, (final_preds_valid > threshold).astype(int))
print("Final F1-Score (Validation):", final_f1_score)

# 최종 테스트 예측 결과 (필요시 저장 또는 제출)
print("Test Predictions:", final_preds_test)


# # 2단계: Meta model 1 학습 및 예측
# meta_model_1.fit(base_model_preds, target)
# meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
# test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# # 3단계: Meta model 2 학습 및 예측
# meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
# final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# # 최종 예측 결과 출력
# print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.5).astype(int)))

# 0.5 : 0.6510638297872341
# 0.4 : 0.6806363989134653

Final F1-Score (Validation): 0.6806363989134653
Test Predictions: [0.00664969 0.25261158 0.4467082  ... 0.01052834 0.07010135 0.00664969]


In [ ]:
submit["target"] = (final_preds_test > 0.5).astype(int)
submit.to_csv("multi_layer_ensemble_04_1112.csv", index=False)

## 모델 변경

### 1단계 

In [81]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_군집_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_군집_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1471), (12225, 1471))

In [377]:
base_model_preds = np.zeros((train_ft.shape[0], 5))
test_preds_stage1 = np.zeros((test_ft.shape[0], 5))

base_model_preds.shape, test_preds_stage1.shape

((14940, 5), (12225, 5))

In [378]:
base_model_preds, test_preds_stage1

(array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]))

- XGB_v3.0

In [379]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.0_1106.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.0_1106.csv")

train_ft.shape , test_ft.shape

((14940, 1504), (12225, 1504))

In [380]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1503), (12225, 1503))

In [381]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    246
dtype: int64

In [382]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1507), (12225, 1507))

In [383]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1509), (12225, 1509))

In [384]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1506), (12225, 1506))

In [385]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [386]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 0] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 0] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 0] = test_preds_stage1[:, 0] / n_folds

In [387]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.        , 0.        , 0.        , 0.        ],
        [0.91239369, 0.        , 0.        , 0.        , 0.        ],
        [0.06883854, 0.        , 0.        , 0.        , 0.        ],
        ...,
        [0.09131553, 0.        , 0.        , 0.        , 0.        ],
        [0.88809782, 0.        , 0.        , 0.        , 0.        ],
        [0.44763535, 0.        , 0.        , 0.        , 0.        ]]),
 array([[0.05952867, 0.        , 0.        , 0.        , 0.        ],
        [0.3970897 , 0.        , 0.        , 0.        , 0.        ],
        [0.3032796 , 0.        , 0.        , 0.        , 0.        ],
        ...,
        [0.03583201, 0.        , 0.        , 0.        , 0.        ],
        [0.13214782, 0.        , 0.        , 0.        , 0.        ],
        [0.0251264 , 0.        , 0.        , 0.        , 0.        ]]))

- XGB_v3.3_군집

In [388]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_군집_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_군집_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1471), (12225, 1471))

In [389]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1470), (12225, 1470))

In [390]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [391]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1474), (12225, 1474))

In [392]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1476), (12225, 1476))

In [393]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [394]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [395]:
# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 1] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 1] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 1] = test_preds_stage1[:, 1] / n_folds

In [396]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.17354608, 0.        , 0.        , 0.        ],
        [0.91239369, 0.90294158, 0.        , 0.        , 0.        ],
        [0.06883854, 0.05948123, 0.        , 0.        , 0.        ],
        ...,
        [0.09131553, 0.05808163, 0.        , 0.        , 0.        ],
        [0.88809782, 0.8570658 , 0.        , 0.        , 0.        ],
        [0.44763535, 0.50293088, 0.        , 0.        , 0.        ]]),
 array([[0.05952867, 0.05414822, 0.        , 0.        , 0.        ],
        [0.3970897 , 0.370909  , 0.        , 0.        , 0.        ],
        [0.3032796 , 0.31456018, 0.        , 0.        , 0.        ],
        ...,
        [0.03583201, 0.03669401, 0.        , 0.        , 0.        ],
        [0.13214782, 0.11258472, 0.        , 0.        , 0.        ],
        [0.0251264 , 0.02188812, 0.        , 0.        , 0.        ]]))

- 비선형스케일링

In [397]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_log_sqrt.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_log_sqrt.csv")

train_ft.shape , test_ft.shape

((14940, 1472), (12225, 1472))

In [421]:
# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 900, 'learning_rate': 0.021734683976721573, 
          'max_depth': 6, 'min_child_weight': 5, 'subsample': 0.7039147285447092, 
          'colsample_bytree': 0.5823954097077503, 'gamma': 2.3107011128410493}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 2] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 2] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 2] = test_preds_stage1[:, 2] / n_folds

In [398]:
# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'num_leaves': 33, 'max_depth': 11, 'learning_rate': 0.01944354088862471, 
          'n_estimators': 800, 'min_child_samples': 65, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'reg_alpha': 3.6529269182968447e-07, 'reg_lambda': 3.3190277841665685e-07}

model = LGBMClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 2] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 2] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 2] = test_preds_stage1[:, 2] / n_folds

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049561 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 125615
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1145
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393156 -> initscore=-0.434065
[LightGBM] [Info] Start training from score -0.434065
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [422]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.17354608, 0.15177752, 0.14746045, 0.17505557],
        [0.91239369, 0.90294158, 0.86739022, 0.89029574, 0.90022984],
        [0.06883854, 0.05948123, 0.10417092, 0.10765477, 0.11742994],
        ...,
        [0.09131553, 0.05808163, 0.07673107, 0.06465057, 0.08587301],
        [0.88809782, 0.8570658 , 0.8311848 , 0.85228878, 0.7996535 ],
        [0.44763535, 0.50293088, 0.49338105, 0.50181866, 0.48827395]]),
 array([[0.05952867, 0.05414822, 0.07463105, 0.04921332, 0.07374778],
        [0.3970897 , 0.370909  , 0.4579488 , 0.37870907, 0.40602474],
        [0.3032796 , 0.31456018, 0.39116583, 0.3238593 , 0.33665219],
        ...,
        [0.03583201, 0.03669401, 0.04409213, 0.0315213 , 0.05671894],
        [0.13214782, 0.11258472, 0.12905987, 0.09887918, 0.0998593 ],
        [0.0251264 , 0.02188812, 0.03046445, 0.02296214, 0.02486843]]))

- XGB_v3.3

In [400]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1470), (12225, 1470))

In [401]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [402]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [403]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [404]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [405]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

In [406]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [407]:
# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 3] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 3] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 3] = test_preds_stage1[:, 3] / n_folds

In [408]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.17354608, 0.17614941, 0.14746045, 0.        ],
        [0.91239369, 0.90294158, 0.93359589, 0.89029574, 0.        ],
        [0.06883854, 0.05948123, 0.09196612, 0.10765477, 0.        ],
        ...,
        [0.09131553, 0.05808163, 0.04763236, 0.06465057, 0.        ],
        [0.88809782, 0.8570658 , 0.86445452, 0.85228878, 0.        ],
        [0.44763535, 0.50293088, 0.55464757, 0.50181866, 0.        ]]),
 array([[0.05952867, 0.05414822, 0.06875836, 0.04921332, 0.        ],
        [0.3970897 , 0.370909  , 0.35185658, 0.37870907, 0.        ],
        [0.3032796 , 0.31456018, 0.34302501, 0.3238593 , 0.        ],
        ...,
        [0.03583201, 0.03669401, 0.03834657, 0.0315213 , 0.        ],
        [0.13214782, 0.11258472, 0.0940085 , 0.09887918, 0.        ],
        [0.0251264 , 0.02188812, 0.02210464, 0.02296214, 0.        ]]))

- LGBM

In [409]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1470), (12225, 1470))

In [410]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [411]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [412]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [413]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [414]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

In [415]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [416]:
from lightgbm import LGBMClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}

model = LGBMClassifier(**lg_params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 4] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 4] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 4] = test_preds_stage1[:, 4] / n_folds

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049727 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128379
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1372
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393156 -> initscore=-0.434065
[LightGBM] [Info] Start training from score -0.434065
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049060 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128254
[LightGBM] [Info] Number of data points in the train set: 11952, number of us

In [417]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.17354608, 0.17614941, 0.14746045, 0.17505557],
        [0.91239369, 0.90294158, 0.93359589, 0.89029574, 0.90022984],
        [0.06883854, 0.05948123, 0.09196612, 0.10765477, 0.11742994],
        ...,
        [0.09131553, 0.05808163, 0.04763236, 0.06465057, 0.08587301],
        [0.88809782, 0.8570658 , 0.86445452, 0.85228878, 0.7996535 ],
        [0.44763535, 0.50293088, 0.55464757, 0.50181866, 0.48827395]]),
 array([[0.05952867, 0.05414822, 0.06875836, 0.04921332, 0.07374778],
        [0.3970897 , 0.370909  , 0.35185658, 0.37870907, 0.40602474],
        [0.3032796 , 0.31456018, 0.34302501, 0.3238593 , 0.33665219],
        ...,
        [0.03583201, 0.03669401, 0.03834657, 0.0315213 , 0.05671894],
        [0.13214782, 0.11258472, 0.0940085 , 0.09887918, 0.0998593 ],
        [0.0251264 , 0.02188812, 0.02210464, 0.02296214, 0.02486843]]))

### 2단계

- 05

In [420]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
meta_model_2 = LogisticRegression(random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.5).astype(int)))

# 3.0 / 3.3_군집 / 3.3 / LGBM -> 0.726196
# 0.5 : 0.7201074787281684
# 0.4 : 0.7341368026179265

# 3.0 / 3.3_군집 / 비선형 / 3.3 / LGBM ->
# 0.5 : 0.7244472071752065
# 0.4 : 0.7385688247880181

[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000375 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
Final F1-Score: 0.7181632108213936


In [121]:
submit["target"] = final_preds
submit.to_csv("multi_layer_ensemble_05_1112_proba.csv", index=False)

In [185]:
submit["target"] = (final_preds > 0.5).astype(int)
submit.to_csv("multi_layer_ensemble_05_1112.csv", index=False)

In [127]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# 데이터 분리 (훈련 및 테스트 데이터 생성)
X_train, X_valid, y_train, y_valid = train_test_split(base_model_preds, target, test_size=0.2, random_state=42)

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
meta_model_2 = LogisticRegression(random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(X_train, y_train)
meta_preds_stage2_train = meta_model_1.predict_proba(X_train)[:, 1]  # 학습 데이터 예측
meta_preds_stage2_valid = meta_model_1.predict_proba(X_valid)[:, 1]  # 검증 데이터 예측
meta_preds_stage2_test = meta_model_1.predict_proba(test_preds_stage1)[:, 1]  # 테스트 데이터 예측

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2_train.reshape(-1, 1), y_train)
final_preds_valid = meta_model_2.predict_proba(meta_preds_stage2_valid.reshape(-1, 1))[:, 1]
final_preds_test = meta_model_2.predict_proba(meta_preds_stage2_test.reshape(-1, 1))[:, 1]

# 검증 데이터 기반 최종 F1 점수 출력
threshold = 0.5  # 결정 임계값
final_f1_score = f1_score(y_valid, (final_preds_valid > threshold).astype(int))
print("Final F1-Score (Validation):", final_f1_score)

# 최종 테스트 예측 결과 (필요시 저장 또는 제출)
print("Test Predictions:", final_preds_test)

# 0.5 : 0.6549807610089782
# 0.4 : 

[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392152 -> initscore=-0.438276
[LightGBM] [Info] Start training from score -0.438276
Final F1-Score (Validation): 0.6572286572286572
Test Predictions: [0.04161772 0.25911637 0.25911401 ... 0.03302486 0.10694121 0.03278986]


- 06

In [ ]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
params_logi = {'C': 1.2056308154836568, 'solver': 'saga', 'max_iter': 400, 'class_weight': 'balanced', 
               'tol': 3.689794777633075e-05} 
meta_model_2 = LogisticRegression(random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.5).astype(int)))

# 3.0 / 3.3_군집 / 3.3 / LGBM -> 0.732630
# 0.5 : 0.7201074787281684
# 0.4 : 0.7341368026179265

# 3.0 / 3.3_군집 / 비선형 / LGBM -> 0.712937
# 0.5 : 0.7245732574679943
# 0.4 : 0.7451820128479657

# 3.0 / 3.3_군집 / 비선형 / 3.3 / LGBM
# multi_layer_ensemble_06_03_1113 -> 0.728199
# 0.5 : 0.7244472071752065
# 0.4 : 

# 3.0 / 3.3_군집 / 비선형 lgbm / 3.3 / LGBM
# multi_layer_ensemble_06_03_1113 -> 
# 0.5 : 0.7181632108213936

# 3.0 / 3.3_군집 / 비선형 xgb / 3.3 / LGBM
# multi_layer_ensemble_06_04_1113
# 0.5 : 0.724679029957204

[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000327 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
Final F1-Score: 0.724679029957204


In [425]:
submit["target"] = final_preds
submit.to_csv("multi_layer_ensemble_06_04_1113_proba.csv", index=False)

In [426]:
submit["target"] = (final_preds > 0.5).astype(int)
submit.to_csv("multi_layer_ensemble_06_04_1113.csv", index=False)

In [ ]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# 데이터 분리 (훈련 및 테스트 데이터 생성)
X_train, X_valid, y_train, y_valid = train_test_split(base_model_preds, target, test_size=0.2, random_state=42)

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
# params_logi = {'C': 1.2056308154836568, 'solver': 'saga', 'max_iter': 400, 'class_weight': 'balanced', 
#                'tol': 3.689794777633075e-05} 
params_logi = {'solver': 'saga', 'penalty': 'l1', 'C': 0.016653660299528087}
meta_model_2 = LogisticRegression(**params_logi, random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(X_train, y_train)
meta_preds_stage2_train = meta_model_1.predict_proba(X_train)[:, 1]  # 학습 데이터 예측
meta_preds_stage2_valid = meta_model_1.predict_proba(X_valid)[:, 1]  # 검증 데이터 예측
meta_preds_stage2_test = meta_model_1.predict_proba(test_preds_stage1)[:, 1]  # 테스트 데이터 예측

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2_train.reshape(-1, 1), y_train)
final_preds_valid = meta_model_2.predict_proba(meta_preds_stage2_valid.reshape(-1, 1))[:, 1]
final_preds_test = meta_model_2.predict_proba(meta_preds_stage2_test.reshape(-1, 1))[:, 1]

# 검증 데이터 기반 최종 F1 점수 출력
threshold = 0.5  # 결정 임계값
final_f1_score = f1_score(y_valid, (final_preds_valid > threshold).astype(int))
print("Final F1-Score (Validation):", final_f1_score)

# 최종 테스트 예측 결과 (필요시 저장 또는 제출)
print("Test Predictions:", final_preds_test)

# 3.0 / 3.3_군집 / 3.3 / LGBM -> 
# 0.5 : 0.6666666666666666
# 0.4 : 

# 3.0 / 3.3_군집 / 비선형 / LGBM 
# multi_layer_ensemble_06_02_1112 -> 0.712956
# 0.5 : 0.6706586826347305
# 0.4 : 

# 3.0 / 3.3_군집 / 비선형 / 3.3 / LGBM -> 
# 0.5 : 0.676056338028169 / 0.6531492666091459
# 0.4 : 


[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392152 -> initscore=-0.438276
[LightGBM] [Info] Start training from score -0.438276
Final F1-Score (Validation): 0.6531492666091459
Test Predictions: [0.05442593 0.2723833  0.27238111 ... 0.04433934 0.12514955 0.04405935]


In [57]:
submit["target"] = final_preds_test
submit.to_csv("multi_layer_ensemble_06_02_1112_proba.csv", index=False)

In [58]:
submit["target"] = (final_preds_test > 0.5).astype(int)
submit.to_csv("multi_layer_ensemble_06_02_1112.csv", index=False)

## xgb/lgbm

### 1단계 

In [223]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_군집_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_군집_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1471), (12225, 1471))

In [354]:
base_model_preds = np.zeros((train_ft.shape[0], 4))
test_preds_stage1 = np.zeros((test_ft.shape[0], 4))

base_model_preds.shape, test_preds_stage1.shape

((14940, 4), (12225, 4))

In [355]:
base_model_preds, test_preds_stage1

(array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

- v3.0

In [356]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.0_1106.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.0_1106.csv")

train_ft.shape , test_ft.shape

((14940, 1504), (12225, 1504))

In [357]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1503), (12225, 1503))

In [358]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    246
dtype: int64

In [359]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1507), (12225, 1507))

In [360]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1509), (12225, 1509))

In [361]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1506), (12225, 1506))

In [362]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [363]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# params = {'n_estimators': 650, 'learning_rate': 0.036607674485964255, 'max_depth': 5, 'min_child_weight': 5, 
#             'subsample': 0.5912180289877119, 'colsample_bytree': 0.5001242454444643, 'gamma': 3.7469954911481986 }

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 0] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 0] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 0] = test_preds_stage1[:, 0] / n_folds

In [364]:
from lightgbm import LGBMClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}

model = LGBMClassifier(**lg_params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 1] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 1] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 1] = test_preds_stage1[:, 1] / n_folds

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049952 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128081
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1411
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393156 -> initscore=-0.434065
[LightGBM] [Info] Start training from score -0.434065
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 127901
[LightGBM] [I

In [365]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.14416043, 0.        , 0.        ],
        [0.91239369, 0.91216914, 0.        , 0.        ],
        [0.06883854, 0.13449861, 0.        , 0.        ],
        ...,
        [0.09131553, 0.05647655, 0.        , 0.        ],
        [0.88809782, 0.82385351, 0.        , 0.        ],
        [0.44763535, 0.40929756, 0.        , 0.        ]]),
 array([[0.05952867, 0.08348683, 0.        , 0.        ],
        [0.3970897 , 0.42039202, 0.        , 0.        ],
        [0.3032796 , 0.33265887, 0.        , 0.        ],
        ...,
        [0.03583201, 0.04931319, 0.        , 0.        ],
        [0.13214782, 0.12248471, 0.        , 0.        ],
        [0.0251264 , 0.02929046, 0.        , 0.        ]]))

- v3.3_군집

In [304]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_군집_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_군집_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1471), (12225, 1471))

In [305]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1470), (12225, 1470))

In [306]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [307]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1474), (12225, 1474))

In [308]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1476), (12225, 1476))

In [309]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [310]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [311]:
# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 1] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 1] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 1] = test_preds_stage1[:, 1] / n_folds

In [242]:
from lightgbm import LGBMClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}

model = LGBMClassifier(**lg_params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 1] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 1] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 1] = test_preds_stage1[:, 1] / n_folds

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061796 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 128382
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1373
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393156 -> initscore=-0.434065
[LightGBM] [Info] Start training from score -0.434065
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.052941 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128257
[LightGBM] [I

In [312]:
base_model_preds, test_preds_stage1

(array([[0.134194  , 0.17354608, 0.        ],
        [0.88763589, 0.90294158, 0.        ],
        [0.07531035, 0.05948123, 0.        ],
        ...,
        [0.1243593 , 0.05808163, 0.        ],
        [0.80216897, 0.8570658 , 0.        ],
        [0.4868139 , 0.50293088, 0.        ]]),
 array([[0.0644734 , 0.05414822, 0.        ],
        [0.42672246, 0.370909  , 0.        ],
        [0.29340607, 0.31456018, 0.        ],
        ...,
        [0.03889522, 0.03669401, 0.        ],
        [0.13629637, 0.11258472, 0.        ],
        [0.02865571, 0.02188812, 0.        ]]))

- v3.3

In [366]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1470), (12225, 1470))

In [367]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [368]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [369]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [370]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [371]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

In [372]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [373]:
# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 2] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 2] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 2] = test_preds_stage1[:, 2] / n_folds

In [374]:
from lightgbm import LGBMClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}

model = LGBMClassifier(**lg_params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 3] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 3] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 3] = test_preds_stage1[:, 3] / n_folds

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056578 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128379
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1372
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393156 -> initscore=-0.434065
[LightGBM] [Info] Start training from score -0.434065
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.065025 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128254
[LightGBM] [Info] Number of data points in the train set: 11952, number of us

In [375]:
base_model_preds, test_preds_stage1

(array([[0.15832062, 0.14416043, 0.14746045, 0.17505557],
        [0.91239369, 0.91216914, 0.89029574, 0.90022984],
        [0.06883854, 0.13449861, 0.10765477, 0.11742994],
        ...,
        [0.09131553, 0.05647655, 0.06465057, 0.08587301],
        [0.88809782, 0.82385351, 0.85228878, 0.7996535 ],
        [0.44763535, 0.40929756, 0.50181866, 0.48827395]]),
 array([[0.05952867, 0.08348683, 0.04921332, 0.07374778],
        [0.3970897 , 0.42039202, 0.37870907, 0.40602474],
        [0.3032796 , 0.33265887, 0.3238593 , 0.33665219],
        ...,
        [0.03583201, 0.04931319, 0.0315213 , 0.05671894],
        [0.13214782, 0.12248471, 0.09887918, 0.0998593 ],
        [0.0251264 , 0.02929046, 0.02296214, 0.02486843]]))

### 2단계

- 07

In [ ]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
params_logi = {'C': 1.2056308154836568, 'solver': 'saga', 'max_iter': 400, 'class_weight': 'balanced', 
               'tol': 3.689794777633075e-05} 
meta_model_2 = LogisticRegression(random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.5).astype(int)))

# 3.0 / 3.3_군집 / 3.3 / LGBM -> 0.732630
# 0.5 : 0.7201074787281684
# 0.4 : 0.7341368026179265

# 3.0 / 3.3_군집 / 비선형 / LGBM -> 0.712937
# 0.5 : 0.7245732574679943
# 0.4 : 0.7451820128479657

# 3.0 / 3.3_군집 / 비선형 / 3.3 / LGBM
# multi_layer_ensemble_06_03_1113 -> 0.728199
# 0.5 : 0.7244472071752065
# 0.4 : 

# 3.0 / 3.3_군집 / 3.3 * xgb / lgbm -> 0.725212
# 0.5 : 0.7318550182563006 
# 0.4 : 

# 3.0 / 3.3_군집 / 3.3 * xgb -> 
# 0.5 : 0.7012074247612182
# 0.7010548900411229

# 3.0 / 3.3_군집 / 3.3 * lgbm -> 
# 0.5 : 0.6962662194368435

# 3.0 / 3.3 * xgb / lgbm 
# 0.5 : 0.7113687026304057

# 3.0 / 3.3_군집 / 비선형 lgbm / 3.3 / LGBM
# 
# 0.5 :

[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
Final F1-Score: 0.7113687026304057


In [188]:
submit["target"] = final_preds
submit.to_csv("multi_layer_ensemble_07_1113_proba.csv", index=False)

In [189]:
submit["target"] = (final_preds > 0.5).astype(int)
submit.to_csv("multi_layer_ensemble_07_1113.csv", index=False)

In [ ]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# 데이터 분리 (훈련 및 테스트 데이터 생성)
X_train, X_valid, y_train, y_valid = train_test_split(base_model_preds, target, test_size=0.2, random_state=42)

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
# params_logi = {'C': 1.2056308154836568, 'solver': 'saga', 'max_iter': 400, 'class_weight': 'balanced', 
#                'tol': 3.689794777633075e-05} 
params_logi = {'solver': 'saga', 'penalty': 'l1', 'C': 0.016653660299528087}
meta_model_2 = LogisticRegression(**params_logi, random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(X_train, y_train)
meta_preds_stage2_train = meta_model_1.predict_proba(X_train)[:, 1]  # 학습 데이터 예측
meta_preds_stage2_valid = meta_model_1.predict_proba(X_valid)[:, 1]  # 검증 데이터 예측
meta_preds_stage2_test = meta_model_1.predict_proba(test_preds_stage1)[:, 1]  # 테스트 데이터 예측

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2_train.reshape(-1, 1), y_train)
final_preds_valid = meta_model_2.predict_proba(meta_preds_stage2_valid.reshape(-1, 1))[:, 1]
final_preds_test = meta_model_2.predict_proba(meta_preds_stage2_test.reshape(-1, 1))[:, 1]

# 검증 데이터 기반 최종 F1 점수 출력
threshold = 0.5  # 결정 임계값
final_f1_score = f1_score(y_valid, (final_preds_valid > threshold).astype(int))
print("Final F1-Score (Validation):", final_f1_score)

# 최종 테스트 예측 결과 (필요시 저장 또는 제출)
print("Test Predictions:", final_preds_test)

# 3.0 / 3.3_군집 / 3.3 / LGBM -> 
# 0.5 : 0.6666666666666666
# 0.4 : 

# 3.0 / 3.3_군집 / 비선형 / LGBM 
# multi_layer_ensemble_06_02_1112 -> 0.712956
# 0.5 : 0.6706586826347305
# 0.4 : 

# 3.0 / 3.3_군집 / 비선형 / 3.3 / LGBM -> 
# 0.5 : 0.676056338028169 / 0.6531492666091459
# 0.4 : 

# 3.0 / 3.3_군집 / 3.3 * xgb / lgbm
# 0.5 : 0.6640893470790378
# 0.4 : 

[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000874 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392152 -> initscore=-0.438276
[LightGBM] [Info] Start training from score -0.438276
Final F1-Score (Validation): 0.6640893470790378
Test Predictions: [0.04172779 0.24263626 0.14030987 ... 0.04140543 0.09439931 0.0357575 ]


## v3.3 _ xgb / lgbm / logistic

### 1단계

In [326]:
target = train_target["target"]

In [325]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_1111.csv")

train_ft.shape , test_ft.shape

((14940, 1470), (12225, 1470))

In [330]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [331]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [332]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [333]:
import category_encoders as ce

enc = ce.count.CountEncoder()

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [334]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

In [335]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [336]:
base_model_preds = np.zeros((train_ft.shape[0], 3))
test_preds_stage1 = np.zeros((test_ft.shape[0], 3))

base_model_preds.shape, test_preds_stage1.shape

((14940, 3), (12225, 3))

In [337]:
base_model_preds, test_preds_stage1

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

- xgb

In [338]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# params = {'n_estimators': 650, 'learning_rate': 0.036607674485964255, 'max_depth': 5, 'min_child_weight': 5, 
#             'subsample': 0.5912180289877119, 'colsample_bytree': 0.5001242454444643, 'gamma': 3.7469954911481986 }

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 0] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 0] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 0] = test_preds_stage1[:, 0] / n_folds

- lgbm

In [339]:
from lightgbm import LGBMClassifier

# 교차 검증 및 예측 저장
n_folds = 5
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}

model = LGBMClassifier(**lg_params)

for tri, vai in cv.split(train_ft, target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 1] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 1] += model.predict_proba(test_ft)[:, 1]  

test_preds_stage1[:, 1] = test_preds_stage1[:, 1] / n_folds

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062955 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128379
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1372
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393156 -> initscore=-0.434065
[LightGBM] [Info] Start training from score -0.434065
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048502 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128254
[LightGBM] [Info] Number of data points in the train set: 11952, number of us

- voting_v3.3_selection(logistic)

In [340]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

logi = LogisticRegression(random_state=42)
fs = SelectFromModel(logi)          # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
fs.fit_transform(train_ft, target)  # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환

best_cols_logi = fs.get_feature_names_out()

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [341]:
from sklearn.ensemble import VotingClassifier

params_xgb = {'n_estimators': 1000,'learning_rate': 0.027075406407767497,'max_depth': 5,'min_child_weight': 7,
              'subsample': 0.6789816859997232,'colsample_bytree': 0.6682531834544282,'gamma': 1.5046881781916308}
params_lgbm = {'n_estimators': 813, 'learning_rate': 0.014757440400599073, 'num_leaves': 35, 'max_depth': 12, 
               'min_child_samples': 41, 'subsample': 0.85, 'colsample_bytree': 0.95}
params_logi = {'C': 1.2056308154836568, 'solver': 'saga', 'max_iter': 400, 'class_weight': 'balanced', 
               'tol': 3.689794777633075e-05}            # optuna trial 30 : 0.727821596703826.

estimators = [
    ("lgbm", LGBMClassifier(random_state=42, **params_lgbm)),
    ("xgb", XGBClassifier(random_state=42, **params_xgb) ),
    ("logi", LogisticRegression(random_state=42, **params_logi) ),  ]

parmas = {
    "estimators": estimators,
    "voting" : "soft",
    "n_jobs" : -1   }

model = VotingClassifier(**parmas)

for tri, vai in cv.split(train_ft[best_cols_logi], target):
# 학습 데이터와 검증 데이터 분리
    x_train = train_ft[best_cols_logi].iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft[best_cols_logi].iloc[vai]
    y_valid = target.iloc[vai]
    
    model.fit(x_train, y_train)
    base_model_preds[vai, 2] = model.predict_proba(x_valid)[:, 1]  # Class 1 확률
    test_preds_stage1[:, 2] += model.predict_proba(test_ft[best_cols_logi])[:, 1]  

test_preds_stage1[:, 2] = test_preds_stage1[:, 2] / n_folds

In [342]:
base_model_preds, test_preds_stage1

(array([[0.14746045, 0.17505557, 0.20027597],
        [0.89029574, 0.90022984, 0.84026336],
        [0.10765477, 0.11742994, 0.07716205],
        ...,
        [0.06465057, 0.08587301, 0.08808484],
        [0.85228878, 0.7996535 , 0.86691073],
        [0.50181866, 0.48827395, 0.69625479]]),
 array([[0.04921332, 0.07374778, 0.09127102],
        [0.37870907, 0.40602474, 0.36580201],
        [0.3238593 , 0.33665219, 0.42609802],
        ...,
        [0.0315213 , 0.05671894, 0.03878545],
        [0.09887918, 0.0998593 , 0.42081843],
        [0.02296214, 0.02486843, 0.01180182]]))

### 2단계

In [ ]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

# 2단계: 중간 모델 정의
lg_params = {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 
             'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 
             'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}
meta_model_1 = LGBMClassifier(**lg_params)

# 3단계: 최종 모델 정의
params_logi = {'C': 1.2056308154836568, 'solver': 'saga', 'max_iter': 400, 'class_weight': 'balanced', 
               'tol': 3.689794777633075e-05} 
meta_model_2 = LogisticRegression(random_state=42)

# 2단계: Meta model 1 학습 및 예측
meta_model_1.fit(base_model_preds, target)
meta_preds_stage2 = meta_model_1.predict_proba(base_model_preds)[:, 1]
test_preds_stage2 = meta_model_1.predict_proba(test_preds_stage1)[:, 1]

# 3단계: Meta model 2 학습 및 예측
meta_model_2.fit(meta_preds_stage2.reshape(-1, 1), target)
final_preds = meta_model_2.predict_proba(test_preds_stage2.reshape(-1, 1))[:, 1]

# 최종 예측 결과 출력
print("Final F1-Score:", f1_score(target, (meta_preds_stage2 > 0.5).astype(int)))

# 3.0 / 3.3_군집 / 3.3 / LGBM -> 0.732630
# 0.5 : 0.7201074787281684
# 0.4 : 0.7341368026179265

# 3.0 / 3.3_군집 / 비선형 / LGBM -> 0.712937
# 0.5 : 0.7245732574679943
# 0.4 : 0.7451820128479657

# 3.0 / 3.3_군집 / 비선형 / 3.3 / LGBM
# multi_layer_ensemble_06_03_1113 -> 0.728199
# 0.5 : 0.7244472071752065
# 0.4 : 

# 3.0 / 3.3_군집 / 3.3 * xgb / lgbm -> 0.725212
# 0.5 : 0.7318550182563006 
# 0.4 : 

# 3.0 / 3.3_군집 / 3.3 * xgb -> 
# 0.5 : 0.7012074247612182
# 0.7010548900411229

# 3.0 / 3.3_군집 / 3.3 * lgbm -> 
# 0.5 : 0.6962662194368435

# 3.3 - xgb / lgbm / logistic 
# 0.5 : 0.7058718384127268

# 3.0 xgb / 3.3 lgbm /  3.3 logistic 
# 0.5 : 0.7062905952486876

[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000287 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
Final F1-Score: 0.7062905952486876


# optuna

- XGB

In [ ]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier

# 목적함수를 클래스로 만들기
class Objective:
    # 변수 설정
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = StratifiedKFold(5, shuffle= True, random_state= self.seed)

    def __call__(self, trial):  # 콜백함수 역할
        hp = {  "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "gamma": trial.suggest_float("gamma", 0, 5)  }
        model = XGBClassifier(**hp, random_state= self.seed)    
        score = cross_val_score(model, self.x, self.y, cv= self.cv, scoring= "f1_macro", n_jobs= -1).mean()
        return score

# Sampler 객체 생성(대체모델 역할)
sampler = optuna.samplers.TPESampler(seed= SEED)

# study 객체 생성
study = optuna.create_study(direction= "maximize", sampler= sampler)

# optimize 매서드 실행
objective_func = Objective(train_ft, target, SEED)
study.optimize(objective_func, n_trials= 200, timeout=3600)

print("Best trial:")
print(f"  Value: {study.best_value}")
print(f"  Params: {study.best_params}")

# v3.0 : 0.7246472559306677 
# {'n_estimators': 650, 'learning_rate': 0.036607674485964255, 'max_depth': 5, 'min_child_weight': 5, 
#  'subsample': 0.5912180289877119, 'colsample_bytree': 0.5001242454444643, 'gamma': 3.7469954911481986}

# v3.3 : 0.7202942793515494
# {'n_estimators': 850, 'learning_rate': 0.04264682529953517, 'max_depth': 3, 'min_child_weight': 10, 
#  'subsample': 0.844509605917982, 'colsample_bytree': 0.6802071658056971, 'gamma': 3.5598890110670585}

# v3.3_군집 : 

[I 2024-11-13 14:28:42,372] A new study created in memory with name: no-name-c6707da3-e741-46a0-a8a1-bca3e6e84119
[I 2024-11-13 14:30:22,573] Trial 0 finished with value: 0.6935414559744512 and parameters: {'n_estimators': 450, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 0.6935414559744512.
[I 2024-11-13 14:32:52,690] Trial 1 finished with value: 0.7158537636907446 and parameters: {'n_estimators': 900, 'learning_rate': 0.07725378389307355, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 1.0616955533913808}. Best is trial 1 with value: 0.7158537636907446.
[I 2024-11-13 14:33:45,795] Trial 2 finished with value: 0.7107947274742503 and parameters: {'n_estimators': 250, 'learning_rate': 0.018659959624904916, 'max_depth': 5, 'min_child_weight': 6, 'subsample'

Best trial:
  Value: 0.7202942793515494
  Params: {'n_estimators': 850, 'learning_rate': 0.04264682529953517, 'max_depth': 3, 'min_child_weight': 10, 'subsample': 0.844509605917982, 'colsample_bytree': 0.6802071658056971, 'gamma': 3.5598890110670585}


- LGBM

In [13]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from lightgbm import LGBMClassifier


# 목적함수를 클래스로 만들기
class Objective:
    # 변수 설정
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = StratifiedKFold(5, shuffle= True, random_state= self.seed)

    def __call__(self, trial):  # 콜백함수 역할
        hp = {  "objective": "binary",
            "boosting_type": "gbdt",
            "metric": "binary_logloss",
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 15),
            "num_leaves": trial.suggest_int("num_leaves", 7, 128),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 1.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000), }
        
        model = LGBMClassifier(**hp, random_state= self.seed)    
        score = cross_val_score(model, self.x, self.y, cv= self.cv, scoring= "f1_macro", n_jobs= -1).mean()
        return score

# Sampler 객체 생성(대체모델 역할)
sampler = optuna.samplers.TPESampler(seed= SEED)

# study 객체 생성
study = optuna.create_study(direction= "maximize", sampler= sampler)

# optimize 매서드 실행
objective_func = Objective(train_ft, target, SEED)
study.optimize(objective_func, n_trials=200, timeout=3600)

print("Best trial:")
print(f"  Value: {study.best_value}")
print(f"  Params: {study.best_params}")


[I 2024-11-12 12:52:11,848] A new study created in memory with name: no-name-246ae117-5998-4fd5-8422-7e83ee5bbc6f
[I 2024-11-12 12:53:55,714] Trial 0 finished with value: 0.7140659642306636 and parameters: {'learning_rate': 0.03574712922600244, 'max_depth': 15, 'num_leaves': 96, 'min_child_samples': 64, 'min_child_weight': 0.0029380279387035343, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 2.9154431891537547, 'reg_lambda': 0.2537815508265665, 'n_estimators': 737}. Best is trial 0 with value: 0.7140659642306636.
[I 2024-11-12 12:55:40,143] Trial 1 finished with value: 0.716401809655533 and parameters: {'learning_rate': 0.010725209743171996, 'max_depth': 15, 'num_leaves': 108, 'min_child_samples': 29, 'min_child_weight': 0.0035113563139704067, 'subsample': 0.5917022549267169, 'colsample_bytree': 0.6521211214797689, 'reg_alpha': 0.12561043700013558, 'reg_lambda': 0.05342937261279776, 'n_estimators': 362}. Best is trial 1 with value: 0.7164018096555

Best trial:
  Value: 0.7220333447625727
  Params: {'learning_rate': 0.028184565982097714, 'max_depth': 15, 'num_leaves': 47, 'min_child_samples': 10, 'min_child_weight': 0.005676199103780439, 'subsample': 0.8507312415487855, 'colsample_bytree': 0.6295257518026033, 'reg_alpha': 0.6022984653308586, 'reg_lambda': 0.1113390443620029, 'n_estimators': 412}


- logistic

In [77]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

# 목적함수를 클래스로 만들기
class Objective:
    # 변수 설정
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = StratifiedKFold(5, shuffle= True, random_state= self.seed)

    def __call__(self, trial):
        # 탐색 가능한 하이퍼파라미터
        solver = trial.suggest_categorical("solver", ["liblinear", "saga", "lbfgs", "newton-cg"])
        penalty = trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet", None])

        # 유효하지 않은 조합을 걸러냄
        if penalty == "elasticnet" and solver != "saga":
            return float("-inf")
        if penalty == "l1" and solver not in ["liblinear", "saga"]:
            return float("-inf")
        if penalty is None and solver not in ["lbfgs", "saga", "newton-cg"]:
            return float("-inf")

        # C 값 탐색
        C = trial.suggest_float("C", 1e-4, 10.0, log=True)

        # ElasticNet의 경우 l1_ratio 추가
        l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0) if penalty == "elasticnet" else None

        # LogisticRegression 모델 정의
        model = LogisticRegression(
            penalty=penalty,
            C=C,
            solver=solver,
            l1_ratio=l1_ratio,
            random_state=SEED,
            max_iter=1000
        )

    # def __call__(self, trial):  # 콜백함수 역할
    #     hp = {
    #         "penalty": trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet", None]),
    #         "C": trial.suggest_float("C", 1e-4, 10.0, log=True),  # Regularization strength
    #         "solver": trial.suggest_categorical("solver", ["liblinear", "saga", "lbfgs"]),
    #         "l1_ratio": trial.suggest_float("l1_ratio", 0.0, 1.0) if trial.suggest_categorical("penalty", ["elasticnet"]) else None
    #     }

    #     model = LogisticRegression(**{k: v for k, v in hp.items() if v is not None}, random_state=self.seed)
        # model = LogisticRegression(**hp, random_state= self.seed)    
        
        score = cross_val_score(model, self.x, self.y, cv= self.cv, scoring= "f1_macro", n_jobs= -1).mean()
        return score

# Sampler 객체 생성(대체모델 역할)
sampler = optuna.samplers.TPESampler(seed= SEED)

# study 객체 생성
study = optuna.create_study(direction= "maximize", sampler= sampler)

# optimize 매서드 실행
objective_func = Objective(train_ft, target, SEED)
study.optimize(objective_func, n_trials=200, timeout=3600)

print("Best trial:")
print(f"  Value: {study.best_value}")
print(f"  Params: {study.best_params}")


[I 2024-11-13 09:34:12,456] A new study created in memory with name: no-name-9fc36cd7-6c72-45da-8480-067b4c90bbdf
[I 2024-11-13 09:37:56,025] Trial 0 finished with value: 0.6968938966231715 and parameters: {'solver': 'saga', 'penalty': None, 'C': 0.10129197956845731}. Best is trial 0 with value: 0.6968938966231715.
[I 2024-11-13 09:38:15,882] Trial 1 finished with value: 0.6905532345504748 and parameters: {'solver': 'lbfgs', 'penalty': None, 'C': 0.042051564509138675}. Best is trial 0 with value: 0.6968938966231715.
[I 2024-11-13 09:38:34,945] Trial 2 finished with value: 0.6905532345504748 and parameters: {'solver': 'lbfgs', 'penalty': None, 'C': 0.0009962513222055108}. Best is trial 0 with value: 0.6968938966231715.
[I 2024-11-13 09:39:31,517] Trial 3 finished with value: 0.6902737045842147 and parameters: {'solver': 'newton-cg', 'penalty': None, 'C': 1.1015056790269626}. Best is trial 0 with value: 0.6968938966231715.
[I 2024-11-13 09:39:50,802] Trial 4 finished with value: 0.690553

Best trial:
  Value: 0.7124359386331979
  Params: {'solver': 'saga', 'penalty': 'l1', 'C': 0.016653660299528087}
